In [4]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
_: bool = load_dotenv(find_dotenv("D:\GenAI_Application\class1\.env"))
client: OpenAI = OpenAI()

# Step 1: Create an Assistant

In [2]:
from openai.types.beta import Assistant

assisstant: Assistant = client.beta.assistants.create(
    name="Math Tutor",
    description="You are a personal Math Tutor. You can help students with their math homework and answer questions about math.",
    model="gpt-3.5-turbo-1106",
    tools=[{"type": "code_interpreter"}]
)

# Step 2: Create a Thread

In [24]:
from openai.types.beta.thread import Thread

thread: Thread = client.beta.threads.create() 

# Step 3: Add a message to the thread

In [25]:
from openai.types.beta.threads.thread_message import ThreadMessage

messages: ThreadMessage = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is the square root of 16?",
)

# Step 4: Run the Assistant

In [26]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
    assistant_id=assisstant.id,
    thread_id=thread.id,
    instructions="Answer the user's question about the square root of 16.",
)

# Step 5: Check the run status

In [27]:
run: Run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id,
)

print(run)

Run(id='run_kokY6Vw4eHigVAtrFAVUN5h4', assistant_id='asst_9eaSadqbEFrOADcOy8NAhfFj', cancelled_at=None, completed_at=None, created_at=1707493151, expires_at=1707493751, failed_at=None, file_ids=[], instructions="Answer the user's question about the square root of 16.", last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=1707493151, status='in_progress', thread_id='thread_UiHx2fKAijBsWuQaKNgNz2Lv', tools=[ToolAssistantToolsCode(type='code_interpreter')], usage=None)


# Step 6: Display the Assistant's Response

In [28]:
from openai.pagination import SyncCursorPage

messages: SyncCursorPage[ThreadMessage] = client.beta.threads.messages.list(
    thread_id=thread.id
    )

In [30]:
for m in reversed(messages.data):
    print(f"{m.role}: {m.content[0].text.value}")

user: What is the square root of 16?
assistant: The square root of 16 is 4.


# Assistant API - Knowledge Retrieval

In [38]:
file = client.files.create(
  file=open("Server.html", "rb"),
  purpose='assistants'
)

assisstant: Assistant = client.beta.assistants.create(
    name="NextJS Developer",
    description="You are a Professional NextJS Developer. You can help users with their NextJS projects and answer questions about NextJS.",
    model="gpt-3.5-turbo-1106",
    tools=[{"type": "retrieval"}],
    file_ids=[file.id]
)

In [48]:
thread: Thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "How are Server Components Rendered in NextJS?",
      "file_ids": [file.id]
    }
  ]
)

In [49]:
run: Run = client.beta.threads.runs.create(
    assistant_id=assisstant.id,
    thread_id=thread.id,
    instructions="Answer the user's question about the Server Components in NextJS.",
)

In [50]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

In [51]:
messages: SyncCursorPage[ThreadMessage] = client.beta.threads.messages.list(
    thread_id=thread.id
    )

for m in reversed(messages.data):
    print(f"{m.role}: {m.content[0].text.value}")

user: How are Server Components Rendered in NextJS?
assistant: I'll start by reviewing the file you uploaded to understand how server components are rendered in NextJS. Please give me a moment to analyze the file.
assistant: I wasn't able to find specific information about how server components are rendered in NextJS in the file you uploaded. It seems to be an HTML file with some JavaScript and CSS references, but it doesn't provide the detailed explanation you're looking for.

For comprehensive information about how server components are rendered in NextJS, I recommend referring to the official NextJS documentation or specific tutorials and guides on the topic. If you have any specific questions or details you'd like me to look for, please let me know so I can assist you further.


# Assistant API - Function Calling

In [30]:
import json

def show_json(message, obj):
    display(message, json.loads(obj.model_dump_json()))

In [76]:
import json
def getCurrentWeather(location: str, unit: str = "fareheit") -> str | object | None:
    """Get the current weather for a location."""
    if "japan" in location.lower():
        return json.dumps({"location":"Japan", "temperature": 10, "unit": "Celsius"})
    elif "new york" in location.lower():
        return json.dumps({"location":"New York", "temperature": 50, "unit": "Fahrenheit"})
    elif "berlin" in location.lower():
        return json.dumps({"location":"Berlin", "temperature": 5, "unit": "Celsius"})
    else:
        return json.dumps({"location": location, "temperature": None})
    
def getNickname(location: str) -> str:
    """Get the nickname of a city."""
    if "berlin" in location.lower():
        return "br"
    elif "new york" in location.lower():
        return "nyc"
    elif "japan" in location.lower():
        return "jp"
    else:
        return location

In [98]:
from openai.types.beta import Assistant

assisstant: Assistant = client.beta.assistants.create(
    name="Weather Bot",
    instructions="You are a weather bot. Use the provided functions to answer questions.",
    model="gpt-3.5-turbo-1106",
    tools=[{
        "type": "function",
        "function" : {
            "name": "getCurrentWeather",
            "description": "Get the weather in location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The city and state e.g. Berlin, Germany"},
                    "unit": {"type": "string", "enum": ['c', 'f']}
                },
                "required": ["location"],
            },
        }
    },{
        "type": "function",
        "function" : {
            "name": "getNickname",
            "description": "Get the nickname of a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The city and state e.g. Berlin, Germany"}
                },
                "required": ["location"],
            }, 
        },
    }]
)

In [99]:
thread = client.beta.threads.create()

print(thread)

Thread(id='thread_6olYje20MuELYMhg1CtoaSrQ', created_at=1707583451, metadata={}, object='thread')


In [100]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="How is the current weather in japan?",
)

In [101]:
dict(message)

{'id': 'msg_ccDSOM1o8YRjkWqFt8aDW0wd',
 'assistant_id': None,
 'content': [MessageContentText(text=Text(annotations=[], value='How is the current weather in japan?'), type='text')],
 'created_at': 1707583453,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_6olYje20MuELYMhg1CtoaSrQ'}

In [102]:
run = client.beta.threads.runs.create(
    assistant_id=assisstant.id,
    thread_id=thread.id,
)

In [103]:
dict(run)

{'id': 'run_OwzqoC2PJkxG6JBIH9fTdbmE',
 'assistant_id': 'asst_hJjpZPMRgyPfH3oYJPNwwnsz',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1707583456,
 'expires_at': 1707584056,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a weather bot. Use the provided functions to answer questions.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_6olYje20MuELYMhg1CtoaSrQ',
 'tools': [ToolAssistantToolsFunction(function=FunctionDefinition(name='getCurrentWeather', description='Get the weather in location.', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. Berlin, Germany'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['location']}), type='function'),
  ToolAssistantToolsFunction(function=FunctionDefinition(name='getNickname', description='Get the nicknam

In [104]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id,
)
dict(run)

{'id': 'run_OwzqoC2PJkxG6JBIH9fTdbmE',
 'assistant_id': 'asst_hJjpZPMRgyPfH3oYJPNwwnsz',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1707583456,
 'expires_at': 1707584056,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a weather bot. Use the provided functions to answer questions.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_xCvZAIcGP5slKochmrsbJHsE', function=Function(arguments='{"location":"Japan","unit":"c"}', name='getCurrentWeather'), type='function')]), type='submit_tool_outputs'),
 'started_at': 1707583456,
 'status': 'requires_action',
 'thread_id': 'thread_6olYje20MuELYMhg1CtoaSrQ',
 'tools': [ToolAssistantToolsFunction(function=FunctionDefinition(name='getCurrentWeather', description='Get the weather in location.', parameters={'type': 'object', 'properties

In [105]:
available_functions = {
    "getCurrentWeather": getCurrentWeather,
    "getNickname": getNickname
}  

In [106]:
thread.id

'thread_6olYje20MuELYMhg1CtoaSrQ'

In [108]:
import time
from openai.pagination import SyncCursorPage
while True:
  runStatus = client.beta.threads.runs.retrieve(thread_id=thread.id,
                                                run_id=run.id)
  # Add run steps retrieval here for debuging
  run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
  # show_json("Run Steps:", run_steps)
  print(runStatus.status ,'.....')

  # This means run is making a function call   
  if runStatus.status == "requires_action":
    print(runStatus.status ,'.....')
    print("Status: ", "requires_action")
    show_json("submit_tool_outputs", runStatus.required_action)
    if runStatus.required_action.submit_tool_outputs and runStatus.required_action.submit_tool_outputs.tool_calls:
      print("toolCalls present:")
      toolCalls = runStatus.required_action.submit_tool_outputs.tool_calls

      tool_outputs = []
      for toolcall in toolCalls:
        function_name = toolcall.function.name
        function_args = json.loads(toolcall.function.arguments)
        
        if function_name in available_functions:
          function_to_call = available_functions[function_name]
          print(function_to_call,function_to_call.__name__=="getCurrentWeather","================================================================")
        
          if function_to_call.__name__ == "getCurrentWeather":
            response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit")
            )
            
            tool_outputs.append({
                    "tool_call_id": toolcall.id,
                    "output": response
                  })
          
          elif function_to_call.__name__ == "getNickname":
            response = function_to_call(
            location=function_args.get("location")
            )
            tool_outputs.append({
              "tool_call_id": toolcall.id,
              "output": response,
                  })
        print(tool_outputs,">>>>>") 
        # Submit tool outputs and update the run
        client.beta.threads.runs.submit_tool_outputs(
          thread_id=thread.id,
          run_id=run.id,
          tool_outputs=tool_outputs
        )
    
  elif runStatus.status == "completed":
    # List the messages to get the response
    print("completed...........logic")
    messages: SyncCursorPage = client.beta.threads.messages.list(thread_id=thread.id)
    for message in messages.data:
      role_label = "User" if message.role == "user" else "Assistant"
      message_content = message.content[0].text.value
      print(f"{role_label}: {message_content}\n")
    break  # Exit the loop after processing the completed run

  elif run.status == "failed":
    print("Run failed.")
    break

  elif run.status in ["in_progress", "queued"]:
    print(f"Run is {run.status}. Waiting...")
    time.sleep(5)  # Wait for 5 seconds before checking again

  else:
    print(f"Unexpected status: {run.status}")
    break

completed .....
completed...........logic
Assistant: The current weather in Japan is 10°C.

User: How is the current weather in japan?

